In [4]:
import pandas as pd
import numpy as np
import ccxt
import pandas_ta as ta
# import plotly as plt
# from matplotlib import pyplot as plt
import plotly.graph_objects as go
# import vectorbt as vb
from datetime import datetime
import time
import math
from binance.client import Client

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None 

In [5]:
api = "8S5aZO9eTTypPrmMvds90p44GWcTHuvEYP4oZozIgWDe9bvyK4JTBkCpEtmrl5Zw"
private = "JPpw6lG5cnBht5ENKUemq4UXGVxS58dHAdkwfOdMjQBzy5Jv1KUl8Zq7tVv4lLqp"

In [18]:
profits = []
count = 0
target_symbol = ["BTC", "ETH", "ADA", "BNB"]
smooth = 2
time_frames = ["1d", "12h", "6h", "4h", "2h", "1h"]
res = []
for timeframe in time_frames:
    time.sleep(30)
    for short_period in range(5, 20):
        for long_period in range(90,110):
    
    # short_period = 10
    # long_period = 100
            # print(client.KLINE_INTERVAL_12HOUR)
            for coin in target_symbol:
                client = Client("YjY0w2BJZZ35o2Q9So1OvRP1gm3exc01LKp2yyLpMth5xmqVn0tQosRYOPrnb7la", "5rRXJ2rOa96hVPZTTOfVP9H4blP6AMt2QN2M0RWX2cMq9PngbJ5MEK88eGB9I4YD")
                start_str="1 year ago UTC"
                bars = client.get_historical_klines(symbol=f'{coin}USDT',interval=timeframe,start_str=start_str)
                test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
                test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
                test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
                test_df["open"] = pd.to_numeric(test_df["open"])
                test_df["high"] = pd.to_numeric(test_df["high"])
                test_df["low"] = pd.to_numeric(test_df["low"])
                test_df["close"] = pd.to_numeric(test_df["close"])
                test_df["volume"] = pd.to_numeric(test_df["volume"])

                if (len(test_df) < long_period):
                    count += 1
                    continue

                test_df["short EMA"] = test_df["close"].copy()
                for i in range(1, len(test_df)):
                    test_df["short EMA"][i] = test_df["short EMA"][i - 1] * (1 - smooth / (short_period + 1)) + test_df["short EMA"][i] * smooth / (short_period + 1)
                    # test_df["short EMA"][i] = test_df["short EMA"][i - 1] * (1 - smooth / (short_period + 1)) + test_df["close"][i] * smooth / (short_period + 1)

                # test_df["short DEMA"] = 2 * test_df["short EMA"].copy()
                test_df["short DEMA"] = test_df["short EMA"].copy()
                # short_DEMA = test_df["short EMA"]
                # for i in range(1, len(test_df)):
                    # short_DEMA[i] = test_df["short EMA"][i - 1] * (1 - smooth / (short_period + 1)) + test_df["short EMA"][i] * smooth / (short_period + 1)
                    # short_DEMA[i] = test_df["short EMA"][i - 1] * (1 - smooth / (short_period + 1)) + test_df["close"][i] * smooth / (short_period + 1)
                # test_df["short DEMA"] -= short_DEMA
                # test_df["short DEMA"] = short_DEMA

                test_df["long EMA"] = test_df["close"].copy()
                for i in range(1, len(test_df)):
                    test_df["long EMA"][i] = test_df["long EMA"][i - 1] * (1 - smooth / (long_period + 1)) + test_df["long EMA"][i] * smooth / (long_period + 1)
                    # test_df["long EMA"][i] = test_df["long EMA"][i - 1] * (1 - smooth / (long_period + 1)) + test_df["close"][i] * smooth / (long_period + 1)

                # test_df["long DEMA"] = 2 * test_df["long EMA"].copy()
                test_df["long DEMA"] = test_df["long EMA"].copy()
                # long_DEMA = test_df["long EMA"]
                # for i in range(1, len(test_df)):
                    # long_DEMA[i] = test_df["long EMA"][i - 1] * (1 - smooth / (long_period + 1)) + test_df["long EMA"][i] * smooth / (long_period + 1)
                    # long_DEMA[i] = test_df["long EMA"][i - 1] * (1 - smooth / (long_period + 1)) + test_df["close"][i] * smooth / (long_period + 1)
                # test_df["long DEMA"] -= long_DEMA
                # test_df["long DEMA"] = long_DEMA


                money = 1000
                storage = 0

                # test_df["close"][150:].plot()
                # test_df["short DEMA"][150:].plot()
                # if (150.0 >= len(test_df) // 2):
                #     continue
                # for i in range(2, len(test_df) // 2):
                # for i in range(len(test_df) // 2, len(test_df)):
                for i in range(150, len(test_df)):
                # for i in range(500, 2000):
                # for i in range(len(test_df) - 3 * 184, len(test_df)):
                    if (test_df["short DEMA"][i - 2] < test_df["long DEMA"][i - 2] and test_df["short DEMA"][i - 1] > test_df["long DEMA"][i - 1]):
                        if (storage < 0):
                            money += storage * test_df["close"][i]
                            storage = 0
                        storage += money / test_df["close"][i]
                        money = 0
                    elif (test_df["short DEMA"][i - 2] > test_df["long DEMA"][i - 2] and test_df["short DEMA"][i - 1] < test_df["long DEMA"][i - 1]):
                        if (storage > 0):
                            money += storage * test_df["close"][i]
                            storage = 0
                        storage -= money / test_df["close"][i]
                        money -= storage * test_df["close"][i]


                # print(coin, (money + storage * test_df["close"][len(test_df) - 1]) / 1000)  
                profits.append((money + storage * test_df["close"][len(test_df) - 1]) / 1000)
            average = (sum(profits)) /  len(profits)
            print("short", short_period, "long", long_period, "timeframe", timeframe, "average", average)
            res.append([short_period, long_period, timeframe, average])


short 5 long 90 timeframe 1d average 1.5287955015195434
short 5 long 91 timeframe 1d average 1.542339359478725
short 5 long 92 timeframe 1d average 1.5477282988564118
short 5 long 93 timeframe 1d average 1.5504160428904605
short 5 long 94 timeframe 1d average 1.5552109336383584
short 5 long 95 timeframe 1d average 1.5579994541107371
short 5 long 96 timeframe 1d average 1.5570585307247111
short 5 long 97 timeframe 1d average 1.5592924438851756
short 5 long 98 timeframe 1d average 1.5640556912250423
short 5 long 99 timeframe 1d average 1.5630367073317104
short 5 long 100 timeframe 1d average 1.5622017668163202
short 5 long 101 timeframe 1d average 1.5632080273587279
short 5 long 102 timeframe 1d average 1.5621919721144064
short 5 long 103 timeframe 1d average 1.5618576922210472
short 5 long 104 timeframe 1d average 1.5615590713042848
short 5 long 105 timeframe 1d average 1.561338823888225
short 5 long 106 timeframe 1d average 1.5611442363127772
short 5 long 107 timeframe 1d average 1.560

KeyboardInterrupt: 